##Summary


* Import Libraries
* Load and Preview Data from AWS Data Lake
* Data Wrangling
  * String Mapping
  * Use Chi-Square to Select Top 1000 Features
  * Reshape and split the data 
* Deep Learning Architecture
  * Set the Hyperparameters
  * Model Construction
	* Learning Rate / Adam Decay Tuning Based on Gaussian Process
	* Bayesian Optimization
	* Best Model for Training
* Model Evaluation and Model Fitting

## Potential Application of Deep Learning using TCGA
* ### Identify potential tissue of origin of unknown cancer types by molecular profiles
* ### Predict tissue of origin of metastatic cancer

Large-scale cancer genomics data often imposes great challenges in terms of computational algorithms. The high dimensioanl dataset is suitable for applying a deep learning algorithm. Given the features and labels, models can be trained to classify future samples based on similar gene expression.

## RNA-Seq (HiSeq) PANCAN Data Set

Data Abstract:

This collection of data is part of the RNA-Seq (HiSeq) PANCAN data set. It is an extraction of gene expressions of patients having different types of tumor: BRCA, KIRC, COAD, LUAD and PRAD. The data source is UCI Machine Learning Repository https://archive.ics.uci.edu/ml/datasets/gene+expression+cancer+RNA-Seq

## Import Libraries

Keras API will be used to conduct multilayer deep learning models. Scikit-Optimize is used to optimize the best parameters. And plotly is imported to do vivid and interactive data visualization.

In [8]:
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer, OneHotEncoder, StandardScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics
import plotly.graph_objs as go
from collections import Counter
import numpy as np
import pandas as pd
import os
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from plotly.subplots import make_subplots

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import LSTM, Embedding, Flatten
from keras import optimizers, regularizers
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

import skopt
from skopt import gbrt_minimize, gp_minimize
from skopt.utils import use_named_args
from skopt.space import Real, Categorical, Integer  

import tensorflow
from tensorflow.python.keras import backend as K

Using TensorFlow backend.
/databricks/python/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 from collections import Mapping, MutableMapping

## Load and Preview Data from AWS Data Lake

*Data Loading*

Datasets were stored in our Gold Bucket, AWS Data Lake. Feature data and label data were separately stored.

In [12]:
# Get access to AWS data lake
ACCESS_KEY= os.environ['AWS_ACCESS_KEY']
ENCODED_SECRET_KEY = os.environ['AWS_SECRET_ACCESS_KEY'].replace('/', '%2F')

GOLD_BUCKET = 'oculadata-gold'
MOUNT_GOLD = '/mnt/gold'

mounted_paths = map(lambda m: m.mountPoint, dbutils.fs.mounts())

# Mount the data
if MOUNT_GOLD not in mounted_paths:
  dbutils.fs.mount(f's3a://{ACCESS_KEY}:{ENCODED_SECRET_KEY}@{GOLD_BUCKET}', MOUNT_GOLD)
  
print(dbutils.fs.mounts())

project = 'PanCanAtlas'
filepath = f'{MOUNT_GOLD}/{project}'

infer_schema = 'true'
if_header = 'true'

[MountInfo(mountPoint='/databricks-datasets', source='databricks-datasets', encryptionType='sse-s3'), MountInfo(mountPoint='/mnt/gold', source='s3a://oculadata-gold', encryptionType=''), MountInfo(mountPoint='/databricks/mlflow-tracking', source='databricks/mlflow-tracking', encryptionType='sse-s3'), MountInfo(mountPoint='/databricks-results', source='databricks-results', encryptionType='sse-s3'), MountInfo(mountPoint='/mnt/mount-oculadatalanding', source='s3a://oculadatalanding', encryptionType=''), MountInfo(mountPoint='/databricks/mlflow-registry', source='databricks/mlflow-registry', encryptionType='sse-s3'), MountInfo(mountPoint='/', source='DatabricksRoot', encryptionType='sse-s3')]

In [13]:
# get the feature data
tablename = 'Sameple_Cancer_data'
file_location = f'dbfs:{filepath}/{tablename}/'
file_type = 'delta'

r_data = spark.read.option("maxColumns", 25000).format(file_type)\
        .option("header", if_header)\
        .option("inferSchema", infer_schema)\
        .load(file_location)

In [14]:
r_data.limit(10).toPandas()

,_c0,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,gene_9,gene_10,gene_11,gene_12,gene_13,gene_14,gene_15,gene_16,gene_17,gene_18,gene_19,gene_20,gene_21,gene_22,gene_23,gene_24,gene_25,gene_26,gene_27,gene_28,gene_29,gene_30,gene_31,gene_32,gene_33,gene_34,gene_35,gene_36,gene_37,gene_38,...,gene_20491,gene_20492,gene_20493,gene_20494,gene_20495,gene_20496,gene_20497,gene_20498,gene_20499,gene_20500,gene_20501,gene_20502,gene_20503,gene_20504,gene_20505,gene_20506,gene_20507,gene_20508,gene_20509,gene_20510,gene_20511,gene_20512,gene_20513,gene_20514,gene_20515,gene_20516,gene_20517,gene_20518,gene_20519,gene_20520,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
0,sample_0,0.0,2.017209,3.265527,5.478487,10.431999,0.0,7.175175,0.591871,0.0,0.000000,0.591871,1.334282,2.015391,0.591871,0.000000,0.0,0.0,0.000000,0.591871,5.619994,1.334282,0.000000,9.796088,0.0,0.000000,1.598651,7.215116,10.839070,6.620204,9.513538,0.000000,4.063658,7.764805,4.747656,13.714396,10.034496,0.000000,0.0,9.833458,...,9.370304,10.362393,5.589928,8.141964,0.000000,2.736583,7.037152,7.123480,10.967399,5.902800,3.719370,7.203554,6.042557,2.602077,7.425526,7.846957,2.824951,6.239396,0.000000,8.469593,0.0,6.535978,6.968701,7.128881,7.175175,9.249369,7.025970,8.045563,7.475709,7.205236,4.926711,8.210257,9.723516,7.220030,9.119813,12.003135,9.650743,8.921326,5.286759,0.0
1,sample_1,0.0,0.592732,1.588421,7.586157,9.623011,0.0,6.816049,0.000000,0.0,0.000000,0.000000,0.587845,2.466601,1.004394,0.000000,0.0,0.0,0.000000,0.000000,11.055208,3.562621,0.000000,10.070470,0.0,0.000000,0.000000,9.949812,8.522476,1.174790,4.926991,0.000000,0.000000,5.819832,1.327170,13.286240,6.663316,0.587845,0.0,9.533302,...,8.882967,9.898199,7.069401,7.186134,0.000000,3.134993,6.648930,6.715701,9.536238,1.004394,5.555482,8.029260,6.366219,0.811142,7.991732,7.161001,0.000000,4.708877,0.811142,8.451689,0.0,7.242336,8.046284,6.047558,8.572901,7.549030,7.019935,9.458940,9.190867,10.639259,4.593372,7.323865,9.740931,6.256586,8.381612,12.674552,10.517059,9.397854,2.094168,0.0
2,sample_2,0.0,3.511759,4.327199,6.881787,9.870730,0.0,6.972130,0.452595,0.0,0.000000,0.000000,0.452595,1.981122,1.074163,0.000000,0.0,0.0,0.000000,1.683023,8.210248,4.195285,3.660427,8.970920,0.0,0.000000,0.796598,6.096650,9.861616,7.680507,3.119439,0.000000,0.452595,7.899526,0.000000,10.731098,6.967883,0.452595,0.0,9.646323,...,10.355637,10.423274,5.170201,6.194260,0.000000,3.677147,6.271990,7.089816,9.675220,0.000000,4.224017,8.020402,6.967883,5.014445,8.400038,7.527555,0.000000,4.997902,0.796598,7.761132,0.0,6.820460,8.048983,6.661493,7.716332,6.745802,7.524667,8.602350,9.036654,10.336027,5.125213,8.127123,10.908640,5.401607,9.911597,9.045255,9.788359,10.090470,1.683023,0.0
3,sample_3,0.0,3.663618,4.507649,6.659068,10.196184,0.0,7.843375,0.434882,0.0,0.000000,0.000000,0.434882,2.874246,0.000000,0.000000,0.0,0.0,0.000000,1.267356,8.306317,3.573556,0.000000,8.524616,0.0,0.000000,0.000000,3.913761,9.511573,6.469165,7.029895,0.000000,1.267356,6.800641,7.742714,12.659474,8.299890,0.768587,0.0,9.670731,...,10.074382,9.918261,7.117924,7.196145,0.434882,3.609755,8.896696,7.577096,10.731446,5.075383,2.175652,7.675435,6.840816,6.233192,8.899886,8.319085,1.791814,5.661134,1.464093,8.625727,0.0,7.420095,7.784746,7.613915,8.963286,7.744699,7.924997,8.981473,8.665592,9.194823,6.076566,8.792959,10.141520,8.942805,9.601208,11.392682,9.694814,9.684365,3.292001,0.0
4,sample_4,0.0,2.655741,2.821547,6.539454,9.738265,0.0,6.566967,0.360982,0.0,0.000000,0.000000,1.275841,2.141204,0.000000,0.000000,0.0,0.0,0.000000,0.889707,10.149150,2.967630,0.000000,8.047238,0.0,1.435949,0.000000,1.942120,8.821535,5.861429,7.755709,0.000000,0.649386,5.570241,2.612801,13.556734,8.004754,0.000000,0.0,9.587569,...,10.129154,10.062303,6.911620,7.855149,0.360982,3.655810,7.255520,7.292607,10.779793,3.954001,6.991148,8.153248,7.508444,4.586531,9.152227,8.227717,0.360982,6.227104,0.

In [15]:
# get the label data
tablename = 'Sameple_Cancer_labels'
file_location = f'dbfs:{filepath}/{tablename}/'

r_label = spark.read.format(file_type)\
                  .option("header", if_header)\
                  .option("inferSchema", infer_schema)\
                  .option("delimiter", ",")\
                  .load(file_location)

r_label.limit(10).toPandas()

,_c0,Class
0,sample_0,PRAD
1,sample_1,LUAD
2,sample_2,PRAD
3,sample_3,PRAD
4,sample_4,BRCA
5,sample_5,PRAD
6,sample_6,KIRC
7,sample_7,PRAD
8,sample_8,BRCA
9,sample_9,PRAD


## Data Wrangling

String data cannot be directly used in model training. So we need to map it to different dummy varibles, which can be distinguished by the algorithm.

## String Mapping

In [19]:
# NULL value check
r_label.filter('Class is null').show()

+---+-----+
_c0|Class|
+---+-----+
+---+-----+

In [20]:
# category indexing with StringIndexer
indexer = StringIndexer(inputCol = 'Class', outputCol = 'Class_Index') 
indexed_r_label = indexer.fit(r_label).transform(r_label)

# OneHotEncodeer to convert into binary values
encoder = OneHotEncoder(inputCol='Class_Index', outputCol='Class_Vec',dropLast = False)
indexed_r_label = encoder.transform(indexed_r_label)

# split into different columns
indexed_r_label = indexed_r_label.select('Class_Vec').rdd.map(lambda x:x[0].toArray().tolist()).toDF()

In [21]:
# show tranferred binary value
indexed_r_label.limit(10).toPandas()

,_1,_2,_3,_4,_5
0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0
6,0.0,1.0,0.0,0.0,0.0
7,0.0,0.0,0.0,1.0,0.0
8,1.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,1.0,0.0


## Use chi-square to select top 1000 features

In [23]:
# apply SelectKBest class to extract top 1000 best features
bestfeatures = SelectKBest(score_func = chi2, k=1000)
fit = bestfeatures.fit(r_data,indexed_r_label)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(r_data.columns) 

# concat the column name
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  

# select top 1000 best features
Selected_Index = featureScores.nlargest(1000,'Score').index
S_Index = Selected_Index.toPandas().values.flatten().tolist()

r_data_ext = r_data.select(*(r_data1.columns[i] for i in S_Index))
r_data_ext.limit(10).toPandas()

,gene_9176,gene_9175,gene_15898,gene_220,gene_219,gene_15896,gene_18135,gene_15899,gene_12069,gene_15895,gene_13976,gene_16132,gene_12995,gene_3439,gene_19153,gene_5829,gene_450,gene_16169,gene_15591,gene_11903,gene_14114,gene_3737,gene_16259,gene_16130,gene_13639,gene_6937,gene_15589,gene_1858,gene_3813,gene_16105,gene_13818,gene_3524,gene_12881,gene_16156,gene_15894,gene_9232,gene_12848,gene_1510,gene_7965,gene_16392,...,gene_9924,gene_18900,gene_16248,gene_5361,gene_7040,gene_9168,gene_13308,gene_3138,gene_2260,gene_753,gene_15604,gene_5752,gene_9181,gene_8193,gene_7659,gene_11393,gene_6070,gene_11576,gene_1331,gene_17680,gene_6850,gene_8431,gene_1992,gene_10421,gene_10690,gene_8032,gene_8013,gene_16357,gene_4209,gene_3448,gene_6945,gene_894,gene_7398,gene_6355,gene_6735,gene_3537,gene_10363,gene_2729,gene_9502,gene_3962
0,18.525161,17.173570,1.334282,0.591871,0.591871,0.591871,6.878308,0.000000,4.692126,0.000000,12.205063,0.000000,13.186662,3.266292,1.010279,0.000000,0.000000,3.478079,0.000000,0.000000,0.000000,11.057694,0.000000,0.0,0.000000,10.671541,0.000000,0.000000,0.000000,0.000000,2.476226,4.852678,8.522283,0.000000,0.000000,1.598651,8.086513,1.598651,0.000000,3.410884,...,0.591871,3.410884,0.000000,3.603763,11.244798,7.555670,1.010279,1.010279,1.822037,11.260690,0.000000,6.180088,1.010279,0.000000,0.000000,6.351165,12.685279,4.063658,9.635185,0.000000,7.555670,2.824951,6.360101,3.017958,9.960060,1.822037,7.896932,2.015391,5.341922,7.455015,3.105561,5.267892,1.334282,6.974529,2.185898,2.824951,8.091890,8.067547,1.822037,0.591871
1,0.000000,0.000000,13.609213,0.000000,0.000000,13.532125,0.000000,12.695983,0.000000,10.068832,0.000000,0.000000,0.000000,0.323658,0.000000,0.000000,0.000000,0.000000,6.555574,11.328675,0.000000,0.323658,0.000000,0.0,0.323658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.323658,0.000000,0.000000,9.975991,0.000000,0.000000,0.323658,0.587845,0.811142,...,2.006585,0.000000,5.915519,1.004394,3.007680,11.158837,3.293282,0.587845,0.000000,4.903299,4.593372,8.382698,6.204051,0.000000,1.706508,7.623435,2.961846,0.000000,3.499846,0.000000,0.000000,4.330264,9.702685,2.592278,7.892907,1.004394,1.327170,0.323658,1.327170,3.095199,2.762370,0.323658,0.000000,9.239379,0.000000,0.000000,4.736361,0.587845,3.890826,0.000000
2,16.053597,14.818422,1.074163,0.452595,0.000000,1.074163,12.900029,0.000000,14.766151,0.000000,9.000285,0.452595,3.000252,0.000000,0.000000,2.785739,0.000000,0.000000,0.000000,0.000000,0.000000,9.709277,0.000000,0.0,1.507160,4.009723,0.000000,0.000000,0.000000,0.796598,0.000000,2.438799,0.000000,0.000000,0.000000,1.306846,5.324101,0.796598,2.228018,1.306846,...,0.000000,1.981122,0.796598,1.507160,5.823816,3.065210,0.000000,0.000000,0.000000,6.834509,0.796598,2.109829,0.000000,0.000000,1.074163,10.090959,12.097624,0.000000,4.410558,0.000000,9.410458,1.074163,1.074163,3.718120,9.639674,4.981168,10.172915,7.082149,2.706508,0.796598,0.000000,1.306846,0.000000,2.337254,0.000000,0.452595,0.000000,8.113753,0.452595,1.981122
3,18.371794,17.371079,0.434882,0.434882,1.039419,4.216416,13.907304,0.000000,8.653719,0.000000,9.779831,0.000000,14.465586,0.000000,0.768587,1.464093,0.000000,0.768587,0.000000,0.000000,0.434882,10.421613,1.931418,0.0,0.000000,7.128840,0.000000,0.000000,0.000000,0.000000,0.000000,0.434882,1.464093,0.000000,0.000000,1.039419,6.959747,0.434882,1.267356,0.434882,...,0.000000,5.942700,0.000000,5.557180,10.321252,6.422593,0.000000,0.000000,1.039419,11.234009,0.768587,16.445607,0.000000,1.039419,0.000000,5.795128,12.109086,0.000000,8.375187,0.000000,3.439636,1.464093,3.573556,3.656393,10.069665,1.267356,9.323969,10.282660,1.637239,8.867656,2.478532,0.000000,0.000000,6.246623,0.000000,0.434882,5.029501,7.492342,0.768587,0.000000
4,0.000000,1.580097,1.095654,0.000000,0.000000,0.360982,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.360982,0.000000,0.000000,0.000000,0.360982,0.649386,0.000000,0.000000,0.000000,0.360982,0.000000,0.0,0.000000,0.000000,14.975920,0.3609

## Reshape and split the data

In [25]:
# change to pandas dataframe
data = r_data_ext.toPandas()
indexed_label = indexed_r_label.toPandas()
# transfer to numpy
data_np = np.array(data)
indexed_label_np = np.array(indexed_label)

# reshape to fit the LSTM expectation
data_np = data_np.reshape(data_np.shape[0],1,data_np.shape[1])
indexed_label_np = indexed_label_np.reshape(indexed_label_np.shape[0],1,indexed_label_np.shape[1])

In [26]:
# split the data to be 70/30
X_train, X_test, y_train, y_test = train_test_split(data_np, indexed_label_np, test_size=0.3, random_state = 123)

*Traning Data and Test Data Overview*

In [28]:
# tranfer back to label
def pred_trans(x):
  pred_ = ['0'] * len(x)
  for row in range(len(x)):
    if x[row][0] == x[row].max():
       pred_[row] = 'BRCA'
    elif x[row][1] == x[row].max():
       pred_[row] = 'KIRC'
    elif x[row][2] == x[row].max():
       pred_[row] = 'LUAD'
    elif x[row][3] == x[row].max():
       pred_[row] = 'PRAD'
    else:
       pred_[row] = 'COAD'
  return pred_

In [29]:
# converse to 2D for viz
y_train = y_train.reshape(-1,5)
y_test = y_test.reshape(-1,5)

# transfer the training dataset to label
y_train_label = pred_trans(y_train)

# transfer the test dataset to label
y_test_label = pred_trans(y_test)

In [30]:
labels = ['BRCA','KIRC','LUAD','PRAD','COAD']

# Create subplots
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels, values = \
              [y_train_label.count('BRCA'),y_train_label.count('KIRC'),y_train_label.count('LUAD'),y_train_label.count('PRAD'),y_train_label.count('COAD')], name="Training Label"), \
              1, 1)
fig.add_trace(go.Pie(labels=labels, values = \
              [y_test_label.count('BRCA'),y_test_label.count('KIRC'),y_test_label.count('LUAD'),y_test_label.count('PRAD'),y_test_label.count('COAD')], name="Test Label"), \
              1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.6, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Traning Label and Test Label Distribution",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Traning Label', x=0.18, y=0.5, font_size=15, showarrow=False),
                 dict(text='Test Label', x=0.82, y=0.5, font_size=15, showarrow=False)])
fig.show()

The training data and test data share a similar distribution, which is good for our further validation and testing process to diminish the bias underlying samples.

## Deep Learning Architecture

<img src="https://github.com/StacyYin/Data-Visualization-/raw/master/PanCan.jpg" alt="Image" border="0">

### Set the Hyperparameters

Hyperparameters were initiated based on the sample's size and target values.

In [36]:
# Hyperparameter Initialization
Num_Class = 5
Num_Inputs = X_train.shape[2]
Num_Hidden = 500

print('Num_Class: ', Num_Class)
print('\nNum_Inputs: ', Num_Inputs)
print('\nNum_Hidden: ', Num_Hidden)

Num_Class: 5

Num_Inputs: 1000

Num_Hidden: 500

### Model Construction

Basic models can be optimized through different hyperparameters. Here learning rate and adam decay were chosen for the model tuning.

## Learning Rate / Adam Decay Tuning Based on Gaussian Process

Learning Rate can be important factors which decide how soon the model will adapt to the problem. A large learning rate may miss the optimal point for modeling. So here a range of values starting from 1e-5 to 1e-3 were set.
Adam Decay was also experimented with this range.

In [41]:
# set the range of learning rate and adam decay
dim_learning_rate = Real(low=1e-5, high=1e-3, prior='log-uniform',
                         name='learning_rate')
dim_adam_decay = Real(low=1e-5,high=1e-3,name="adam_decay")

# tuning part
dimensions = [dim_learning_rate, dim_adam_decay]
# set the default value for start
default_parameters = [1e-3, 1e-3]

# create the model based on Sequential layers
def create_model(learning_rate, adam_decay):
    model = Sequential()
    
    model.add(LSTM(Num_Hidden, input_shape = (1, Num_Inputs), activity_regularizer = regularizers.l2(1e-5), return_sequences = True))
    model.add(Activation('relu'))
    model.add(Dropout(rate=0.1))

    model.add(Dense(Num_Class))
    model.add(Activation('softmax')) 

    # setup our optimizer and compile
    adam = Adam(lr = learning_rate, decay = adam_decay)
    model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [42]:
@use_named_args(dimensions=dimensions)
def fitness(learning_rate, adam_decay):
  
    model = create_model(learning_rate = learning_rate,
                         adam_decay = adam_decay)
    
    
    # fit the model                  
    b_box = model.fit(x=X_train,
                      y=y_train,
                      epochs = 5,
                      batch_size = 128,
                      validation_split=0.1,
                      )
    # return the validation loss for the last epoch
    loss = b_box.history['val_loss'][-1]

    # delete the Keras model with these hyper-parameters from memory
    del model
    
    # clear the Keras session, otherwise it will keep adding new
    # models to the same TensorFlow graph each time we create
    # a model with a different set of hyper-parameters
    K.clear_session()
    tensorflow.reset_default_graph()
    
    # the optimizer aims for the lowest score
    return loss

#### Bayesian Optimization

Bayesian optimization is an approach to optimizing objective functions. It builds a surrogate for the objective and quantifies the uncertainty in that surrogate using a Bayesian machine learning technique, Gaussian process regression, and then uses an acquisition function defined from this surrogate to decide where to sample.

In [45]:
# Parameters Searching
gp_result = gp_minimize(func = fitness,
                        dimensions = dimensions,
                        n_calls = 15, #minimum value is 12
                        noise = 0.0001,
                        n_jobs = -1,
                        kappa = 5,
                        x0 = default_parameters)

Train on 504 samples, validate on 56 samples
Epoch 1/5
128/504 [======>.......................] - ETA: 7s - loss: 0.0715256/504 [==============>...............] - ETA: 2s - loss: 0.0711384/504 [=====================>........] - ETA: 0s - loss: 0.0715504/504 [==============================] - 4s 7ms/step - loss: 0.0705 - val_loss: 0.0520
Epoch 2/5
128/504 [======>.......................] - ETA: 0s - loss: 0.0699256/504 [==============>...............] - ETA: 0s - loss: 0.0702384/504 [=====================>........] - ETA: 0s - loss: 0.0703504/504 [==============================] - 1s 1ms/step - loss: 0.0698 - val_loss: 0.0515
Epoch 3/5
128/504 [======>.......................] - ETA: 0s - loss: 0.0704256/504 [==============>...............] - ETA: 0s - loss: 0.0709384/504 [=====================>........] - ETA: 0s - loss: 0.0700504/504 [==============================] - 1s 1ms/step - loss: 0.0694 - val_loss: 0.0510
Epoch 4/5
128/504 [======>.......................] - ETA: 0s - loss: 0.0687256/504 [==============>...............] - ETA: 0s - loss: 0.0689384/504 [=====================>........] - ETA: 0s - loss: 0.0694504/504 [==============================] - 1s 1ms/step - loss: 0.0684 - val_loss: 0.0505
Epoch 5/5
128/504 [======>.......................] - ETA: 0s - loss: 0.0678256/504 [==============>...............] - ETA: 0s - loss: 0.0678384/504 [=====================>........] - ETA: 0s - loss: 0.0680504/504 [==============================] - 1s 1ms/step - loss: 0.0677 - val_loss: 0.0501
Train on 504 samples, validate on 56 samples
Epoch 1/5
128/504 [======>.......................] - ETA: 7s - loss: 0.0683256/504 [==============>...............] - ETA: 2s - loss: 0.0678384/504 [=====================>........] - ETA: 0s - loss: 0.0680504/504 [==============================] - 3s 6ms/step - loss: 0.0669 - val_loss: 0.0496
Epoch 2/5
128/504 [======>.......................] - ETA: 0s - loss: 0.0664256/504 [==============>...............] - ETA: 0s - loss: 0.0673384/504 [=====================>........] - ETA: 0s - loss: 0.0673504/504 [==============================] - 1s 1ms/step - loss: 0.0664 - val_loss: 0.0491
Epoch 3/5
128/504 [======>.......................] - ETA: 0s - loss: 0.0664256/504 [==============>...............] - ETA: 0s - loss: 0.0671384/504 [=====================>........] - ETA: 0s - loss: 0.0666504/504 [==============================] - 1s 1ms/step 

#### Best Model for Training

*Optimal Hyperparameters*

In [48]:
# fetch the optimal parameters
optimal_learning_rate = gp_result.x[0] # the optimal gp_result.x[0]  is 0.0001
optimal_Adam_decay =gp_result.x[1] # the optimal gp_result.x[1] is 0.0001

print(optimal_learning_rate)
print(optimal_Adam_decay)

0.0001
0.0001

In [49]:
# create the model framework
model = create_model(optimal_learning_rate,optimal_Adam_decay)

# get the model overview
model.summary()

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3733: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3576: The name tf.log is deprecated. Please use tf.math.log instead.

Model: "sequential_1"
_________________________________________________________________
Layer (type) Output Shape Param # 
=================================================================
lstm_1 (LSTM) (None, 1, 500) 3002000 
_________________________________________________________________
activation_1 (Activation) (None, 1, 500) 0 
_________________________________________________________________
dropout_1 (Dropout) (None, 1, 500) 0 
_________________________________________________________________
dense_1 (Dense) (None, 1, 5) 2505 
_________________________________________________________________
activation_2 (Activation) (None, 1, 5) 0 
=================================================================
Total params: 3,004,505
Trainable params: 3,004,505
Non-trainable params: 0
_________________________________________________________________

## Train the Neural Network with Early Stopping

In [51]:
# show the training result - running 10 epochs with early stopping
history = model.fit(X_train,y_train, epochs = 10, batch_size = 128, validation_split = 0.1,callbacks =[EarlyStopping(monitor='val_acc', mode='max')])

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1020: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3005: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

Train on 504 samples, validate on 56 samples
Epoch 1/10
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:197: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:207: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:216: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.is_variable_initialized instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:223: The name tf.variables_initializer is deprecated. Please use tf.compat.v1.variables_initializer instead.

128/504 [======>.......................] - ETA: 8s - loss: 1.6886 - acc: 0.1562256/504 [==============>...............] - ETA: 3s - loss: 1.5900 - acc: 0.2695384/504 [=====================>........] - ETA: 1s - loss: 1.4945 - acc: 0.4271504/504 [==============================] - 4s 8ms/step - loss: 1.4147 - acc: 0.5377 - val_loss: 0.9886 - val_acc: 0.9286
Epoch 2/10
128/504 [======>.......................] - ETA: 0s - loss: 1.0031 - acc: 0.9531256/504 [==============>...............] - ETA: 0s - loss: 0.8995 - acc: 0.9766384/504 [=====================>........] - ETA: 0s - loss: 0.8506 - acc: 0.9792504/504 [==============================] - 1s 1ms/step - loss: 0.7931 - acc: 0.9821 - val_loss: 0.5408 - val_acc: 0.9821
Epoch 3/10
128/504 [======>.......................] - ETA: 0s - loss: 0.5419 - acc: 1.0000256/504 [==============>...............] - ETA: 0s - loss: 0.5072 - acc: 0.9961384/504 [=====================>........] - ETA: 0s - loss: 0.4686 - acc: 0.9974504/504 [==============================] - 1s 1ms/step - loss: 0.4465 - acc: 0.9960 - val_loss: 0.3118 - val_acc: 0.9821

## Model Evaluation and Prediction

*Model Evaluation*

The whole training data was left 10% for validation, which can be used to evaluate the model performance.

*Validation Loss*

Loss is defined as categorical cross entropy loss, which is a metric for evaluating a good classification.

In [55]:
# display the validation loss as the epochs rise
fig1 = go.Figure()
fig1.add_trace(go.Scatter(y=history.history['val_loss'],
                    mode='lines',
                    name='training_loss'))
annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=2,
                              xanchor='left', yanchor='bottom',
                              text='Validation Loss as Epochs Rise',
                              font=dict(family='Arial',
                                        size=20,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
fig1.update_layout(annotations = annotations)
fig1.update_xaxes(title_text='Epoch')
fig1.update_yaxes(title_text='Loss')
fig1.show()

As epochs rise, the validation loss drops significantly.

*Validation Accuracy* 

Accuracy is calculated by number of correct predictions divided by total number of predictions.

In [58]:
# display the validation accuracy as the epochs rise
fig2 = go.Figure()
fig2.add_trace(go.Scatter(y=history.history['val_acc'],
                    mode='lines+markers',
                    name='training_accuracy'))
annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Validation Accuracy as Epochs Rise',
                              font=dict(family='Arial',
                                        size=20,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
fig2.update_layout(annotations = annotations)
fig2.update_xaxes(title_text='Epoch')
fig2.update_yaxes(title_text='Accuracy')
fig2.update_yaxes(range=[0.8, 1])
fig2.show()

Validation Accuracy achieved a remarkable high level after learning.

*Model Prediction*

In [61]:
# predict the test result
y_test_pred = model.predict(X_test)
# round up to 4 decimals
y_test_pred_r2 = np.around(y_test_pred, decimals = 4)
# transfer back to 2 dimensions
y_test_pred_r2 = y_test_pred_r2.reshape(-1,5)

In [62]:
# show the test result
y_test_pred_pd = pd.DataFrame(y_test_pred_r2, columns=["BRCA", "KIRC", "LUAD","PRAD","COAD"])
y_test_pred_pd.index.name = 'PID'
y_test_pred_pd.head(5)

,BRCA,KIRC,LUAD,PRAD,COAD
PID,,,,,
0,0.0687,0.0657,0.1477,0.0380,0.6799
1,0.0979,0.0346,0.7377,0.0708,0.0590
2,0.0665,0.0132,0.8636,0.0291,0.0275
3,0.0406,0.8731,0.0400,0.0167,0.0295
4,0.1818,0.0315,0.1107,0.6506,0.0254


The model would automatically select the cancer type with the highest probability to be the prediction result for each patient.

## Test Accuracy

In [65]:
# print out the test loss and accuracy
y_test = y_test.reshape(y_test.shape[0],1,y_test.shape[1])
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

32/241 [==>...........................] - ETA: 0s160/241 [==================>...........] - ETA: 0s241/241 [==============================] - 0s 516us/step
Test loss: 0.29311567396543825
Test accuracy: 1.0

## Test Original Label & Test Prediction Label

In [67]:
# transfer the training dataset to label
Test_Prediction = pred_trans(y_test_pred_r2)

# transfer the test dataset to label
Test_Original = pred_trans(y_test)

labels = ['BRCA','KIRC','LUAD','PRAD','COAD']

# Create subplots
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels, values = \
              [Test_Original.count('BRCA'),Test_Original.count('KIRC'), \
               Test_Original.count('LUAD'),Test_Original.count('PRAD'),Test_Original.count('COAD')], name="Test Original"), \
              1, 1)
fig.add_trace(go.Pie(labels=labels, values = \
       [Test_Prediction.count('BRCA'),Test_Prediction.count('KIRC'),\
        Test_Prediction.count('LUAD'),Test_Prediction.count('PRAD'),Test_Prediction.count('COAD')], name="Test Prediction"), \
              1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.6, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Test Original and Test Prediction Distribution",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Test Original', x=0.18, y=0.5, font_size=15, showarrow=False),
                 dict(text='Test Prediction', x=0.82, y=0.5, font_size=15, showarrow=False)])
fig.show()

The prediction amazingly covered the test original label, which approved a highly efficient method to identifying the tumor types.

## Save to Gold Bucket in Delta

A Delta table can easily be modified through inserts, deletes, and merges. In addition, all these modifications can be rolled back to obtain an older version of the Delta Table. That way Delta Lake offers us **flexible storage** and helps us to keep control over the changes in the data.

In [71]:
# transfer back to Spark Dataframe
y_test_pred_sp = spark.createDataFrame(y_test_pred_pd)

# add the PID column
y_test_pred_sp2 = y_test_pred_sp.repartition(1).withColumn("PID",monotonically_increasing_id())
y_test_pred_sp2 = y_test_pred_sp2.select("PID","BRCA","KIRC","LUAD","PRAD","COAD")

In [72]:
write_file_name = 'PanCan_TestPred'
write_location = f'{MOUNT_GOLD}/{write_file_name}'

In [73]:
# write to the gold bucket
y_test_pred_sp2.write.format("delta") \
               .option("overwriteSchema", "true") \
               .mode("overwrite") \
               .save(write_location)

*Load From Gold Bucket*

## Business Value - Running SQL Queries against Delta

In [76]:
db_name = 'PanCan'
spark.sql(f"""
   CREATE DATABASE IF NOT EXISTS {db_name}
   """)

Out[69]: DataFrame[]

In [77]:
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {db_name}.{write_file_name}
    USING DELTA
    LOCATION '{write_location}'
""")

Out[70]: DataFrame[]

In [78]:
test_res = spark.sql(f"""
    SELECT * FROM {db_name}.{write_file_name}
    LIMIT 5
""")
display(test_res)

PID,BRCA,KIRC,LUAD,PRAD,COAD
0,0.0687,0.0657,0.1477,0.038,0.6799
1,0.0979,0.0346,0.7377,0.0708,0.059
2,0.0665,0.0132,0.8636,0.0291,0.0275
3,0.0406,0.8731,0.04,0.0167,0.0295
4,0.1818,0.0315,0.1107,0.6506,0.0254


Running SQL Queries on View

In [80]:
# load delta format tables from gold bucket
test_pred_sp = spark.read.format('delta').load(write_location)

In [81]:
# create a temp table
table_name = 'test_pred'
test_pred_sp.createOrReplaceTempView(table_name)

In [82]:
%sql
-- select specific value range
SELECT *
FROM test_pred
WHERE KIRC > 0.6
ORDER BY KIRC DESC

PID,BRCA,KIRC,LUAD,PRAD,COAD
116,0.0254,0.9167,0.0267,0.0099,0.0213
15,0.0237,0.916,0.0248,0.0116,0.0239
232,0.019,0.9145,0.0304,0.011,0.0251
120,0.0336,0.9026,0.0286,0.0106,0.0246
14,0.0291,0.9024,0.0313,0.0136,0.0236
34,0.032,0.8996,0.0356,0.0132,0.0197
87,0.025,0.8989,0.0358,0.0139,0.0265
86,0.0357,0.898,0.0268,0.0135,0.0261
204,0.0303,0.8977,0.0286,0.017,0.0264
96,0.0306,0.8945,0.0366,0.0144,0.0238


## Traverse Back to Older Timestamp

In [84]:
# get any version or timestamp
y_test_pred_1 = spark.read.format("delta") \
                          .option("timestampAsOf", "2020-07-10") \
                          .load(write_location)

# create a temp table
table_name = 'test_pred_1'
y_test_pred_1.createOrReplaceTempView(table_name)

In [85]:
%sql
-- Different Version of Query on 2020-07-10 00:00:00
SELECT *
FROM test_pred_1
WHERE KIRC > 0.2
ORDER BY KIRC DESC

BRCA,KIRC,LUAD,PRAD,COAD
0.2272,0.2075,0.1933,0.1921,0.1799
0.2267,0.2069,0.1931,0.1926,0.1807
0.2276,0.2065,0.1934,0.192,0.1806
0.2264,0.2064,0.1933,0.1925,0.1813
0.227,0.2064,0.1945,0.192,0.1801
0.2266,0.2063,0.1933,0.1929,0.1809
0.2258,0.2063,0.1933,0.1932,0.1813
0.2268,0.2063,0.1938,0.1929,0.1802
0.2275,0.2062,0.1944,0.1909,0.1809
0.2269,0.2062,0.1941,0.1919,0.1809
